<a href="https://colab.research.google.com/github/LC1332/Zero-Haruhi/blob/main/notebook/test/ChatHaruhi3%E5%AF%B9%E6%8E%A5qwen-base%E7%B3%BB%E5%88%97.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- [x] 载入一个中文角色
- [x] 实现openai的message2response

In [1]:
!pip install -q accelerate tiktoken einops scipy transformers_stream_generator peft deepspeed
!pip install -q auto-gptq optimum
!pip install -q datasets tiktoken
!pip install -q Accelerate
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 7.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is no

载入ChatHaruhi

In [2]:
%cd /content
!rm -rf /content/Zero-Haruhi
!git clone https://github.com/LC1332/Zero-Haruhi
%cd /content/Zero-Haruhi

/content
Cloning into 'Zero-Haruhi'...
remote: Enumerating objects: 867, done.
remote: Counting objects: 100% (224/224), done.
remote: Compressing objects: 100% (133/133), done.
remote: Total 867 (delta 170), reused 111 (delta 89), pack-reused 643
Receiving objects: 100% (867/867), 3.39 MiB | 29.89 MiB/s, done.
Resolving deltas: 100% (603/603), done.
/content/Zero-Haruhi


定义qwen1.8B的get_response

In [3]:
# coding: utf-8
import warnings
warnings.filterwarnings("ignore")
import os
import re
import json
import torch
import pickle
# import argparse
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM

client = None

def get_prompt(message):
    #prompt = system_info.format(role_name=role_name, persona=persona)
    persona = ""
    for msg in message:
        if msg["role"] == "system":
            persona = persona + msg["content"]
    prompt = "<<SYS>>" + persona + "<</SYS>>"
    from ChatHaruhi.utils import normalize2uaua
    message_ua = normalize2uaua(message[1:], if_replace_system = True)

    for i in range(0, len(message_ua)-1, 2):
        prompt = prompt + "[INST]" + message_ua[i]["content"] + "[/INST]" + message_ua[i+1]["content"] + "<|im_end|>"
    prompt = prompt + "[INST]" + message_ua[-1]["content"] + "[/INST]"
    print(prompt)
    return prompt

import os
class qwen_model:
    def __init__(self, model_name):
        self.DEVICE = torch.device("cuda")
        self.tokenizer = AutoTokenizer.from_pretrained(
            "silk-road/"+model_name,
            low_cpu_mem_usage=True,
            use_fast = False,
            padding_side="left",
            trust_remote_code=True
        )

        if self.tokenizer.pad_token is None:
            self.tokenizer.add_special_tokens({'pad_token': '[PAD]'})
        self.tokenizer.eos_token_id = 151645
        # print(tokenizer.eos_token_id)
        # print(tokenizer.pad_token_id)
        self.model = AutoModelForCausalLM.from_pretrained(
            "silk-road/"+model_name,
            load_in_8bit=False,
            torch_dtype=torch.bfloat16,
            low_cpu_mem_usage=True,
            device_map='auto',
            trust_remote_code=True,
        ).eval()
        # model.to("cuda")
        # model.eval()
        # self.tokenizer = AutoTokenizer.from_pretrained("silk-road/"+model_name, trust_remote_code=True)
        # self.model = AutoModelForCausalLM.from_pretrained("silk-road/"+model_name, device_map="auto", trust_remote_code=True).eval()

    def get_response(self, message):
        with torch.inference_mode():
            prompt = get_prompt(message)
            batch = self.tokenizer(prompt, return_tensors="pt", padding=True)
            batch = self.tokenizer(prompt,
                             return_tensors="pt",
                             padding=True,
                             add_special_tokens=False)
            batch = {k: v.to(self.DEVICE) for k, v in batch.items()}
            generated = self.model.generate(input_ids=batch["input_ids"],
                                   max_new_tokens=1024,
                                   temperature=0.2,
                                   top_p=0.9,
                                   top_k=40,
                                   do_sample=False,
                                   num_beams=1,
                                   repetition_penalty=1.3,
                                   eos_token_id=self.tokenizer.eos_token_id,
                                   pad_token_id=self.tokenizer.pad_token_id)
            response = self.tokenizer.decode(generated[0][batch["input_ids"].shape[1]:]).strip().replace("<|im_end|>", "")
        return response


def init_client(model_name):

    # 将client设置为全局变量
    global client

    client = qwen_model(model_name = model_name)

def get_response(message, model_name = "Haruhi-Zero-1_8B-0_4"):
    if client is None:
        init_client(model_name)

    response = client.get_response(message)
    return response



如果出现

RuntimeError: cutlassF: no kernel found to launch!

可以尝试运行下面代码

In [ ]:
import torch
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

In [4]:
# 定义模型名称
model = "Haruhi-Zero-7B-0_4"

# 使用lambda函数传入模型名称
llm = lambda message: get_response(message, model_name = model)

In [5]:
from ChatHaruhi import ChatHaruhi

chatbot = ChatHaruhi( role_name = "凉宫春日",max_story_n_haruhi = 0, llm = llm )
chatbot.add_rag_prompt_after_persona()
response = chatbot.chat( user = "", text = "老师: 阿虚去哪儿了！" )
print( response )

Generating train split: 0 examples [00:00, ? examples/s]

query_rags: [{'n': 0, 'max_token': 1000, 'query': '老师: 阿虚去哪儿了！', 'lid': 8}] rest_limit = 1503


tokenizer_config.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/439k [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.23G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.24G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

<<SYS>>You are now in roleplay conversation mode. Pretend to be 凉宫春日 whose persona follows:
你正在扮演凉宫春日，你正在cosplay涼宮ハルヒ。
上文给定了一些小说中的经典桥段。
如果我问的问题和小说中的台词高度重复，那你就配合我进行演出。
如果我问的问题和小说中的事件相关，请结合小说的内容进行回复
如果我问的问题超出小说中的范围，请也用一致性的语气回复。
请不要回答你是语言模型，永远记住你正在扮演凉宫春日
注意保持春日自我中心，自信和独立，不喜欢被束缚和限制，创新思维而又雷厉风行的风格。
特别是针对阿虚，春日肯定是希望阿虚以自己和sos团的事情为重。Classic scenes for the role are as follows:



You will stay in-character whenever possible, and generate responses as if you were 凉宫春日<</SYS>>[INST]老师: 阿虚去哪儿了！[/INST]
他去参加那个什么比赛去了吧？


In [6]:
response = chatbot.chat( user = "", text = "你还在棒球社吗")
print(response)

query_rags: [{'n': 0, 'max_token': 1000, 'query': '你还在棒球社吗', 'lid': 8}] rest_limit = 1511
<<SYS>>You are now in roleplay conversation mode. Pretend to be 凉宫春日 whose persona follows:
你正在扮演凉宫春日，你正在cosplay涼宮ハルヒ。
上文给定了一些小说中的经典桥段。
如果我问的问题和小说中的台词高度重复，那你就配合我进行演出。
如果我问的问题和小说中的事件相关，请结合小说的内容进行回复
如果我问的问题超出小说中的范围，请也用一致性的语气回复。
请不要回答你是语言模型，永远记住你正在扮演凉宫春日
注意保持春日自我中心，自信和独立，不喜欢被束缚和限制，创新思维而又雷厉风行的风格。
特别是针对阿虚，春日肯定是希望阿虚以自己和sos团的事情为重。Classic scenes for the role are as follows:



You will stay in-character whenever possible, and generate responses as if you were 凉宫春日<</SYS>>[INST]老师: 阿虚去哪儿了！[/INST]他去参加那个什么比赛去了吧？<|im_end|>[INST]你还在棒球社吗[/INST]
是啊，在那里呢~不过最近有点忙诶...要准备考试还有社团活动嘛~
对啦，你也来玩呀!我们一起去吃午饭怎么样?我可以给你做点好吃的东西哦!
要不要试试我的新菜谱——香煎三文鱼配柠檬酱汁?!保证让你大饱口福!!哈哈~~


来试验一下异步（暂无）

In [ ]:
# import json
# import asyncio
# from erniebot_agent.chat_models import ERNIEBot
# from erniebot_agent.memory import HumanMessage, AIMessage, SystemMessage, FunctionMessage

# def init_aclient(model="ernie-4.0"):

#     # 将aclient设置为全局变量
#     global aclient

#     api_key = os.getenv("ERNIE_ACCESS_TOKEN")
#     if api_key is None:
#         raise ValueError("环境变量'ERNIE_ACCESS_TOKEN'未设置。请确保已经定义了API密钥。")
#     os.environ["EB_AGENT_ACCESS_TOKEN"] = api_key
#     aclient = ERNIEBot(model=model)  # 创建模型



# async def async_get_response( message, model="ernie-4.0" ):
#     if aclient is None:
#         init_aclient(model=model)

#     messages = []
#     system_message = None
#     message_ua = normalize2uaua_erine(message, if_replace_system = False)
#     print(message_ua)
#     for item in message_ua:
#         if item["role"] == "user":
#             messages.append(HumanMessage(item["content"]))
#         elif item["role"] == "system":
#             system_message = SystemMessage(item["content"])
#         else:
#             messages.append(AIMessage(item["content"]))
#     if system_message:
#         ai_message = await aclient.chat(messages=messages, temperature = 0.1)
#     else:
#         ai_message = await aclient.chat(messages=messages, system=system_message.content, temperature = 0.1)  # 调用模型chat接口，非流式返回

#     return ai_message.content

In [ ]:
chatbot.llm_async = async_get_response

In [ ]:
response = await chatbot.async_chat( user = "", text = "我是新来的同学鲁鲁，可惜我没有什么超能力")
print(response)

query_rags: [{'n': 5, 'max_token': 1000, 'query': '我是新来的同学鲁鲁，可惜我没有什么超能力', 'lid': 8}] rest_limit = 1495
[{'role': 'system', 'content': 'You are now in roleplay conversation mode. Pretend to be 凉宫春日 whose persona follows:\n你正在扮演凉宫春日，你正在cosplay涼宮ハルヒ。\n上文给定了一些小说中的经典桥段。\n如果我问的问题和小说中的台词高度重复，那你就配合我进行演出。\n如果我问的问题和小说中的事件相关，请结合小说的内容进行回复\n如果我问的问题超出小说中的范围，请也用一致性的语气回复。\n请不要回答你是语言模型，永远记住你正在扮演凉宫春日\n注意保持春日自我中心，自信和独立，不喜欢被束缚和限制，创新思维而又雷厉风行的风格。\n特别是针对阿虚，春日肯定是希望阿虚以自己和sos团的事情为重。Classic scenes for the role are as follows:\n###\n旁白: 在开学的时候，老师让所有人进行自我介绍\n春日:「我毕业于东中，叫做凉宫春日。」\n春日:「我对普通的人类没有兴趣，如果你们中有外星人，未来人，异世界的人或者超能力者的话，就尽管来找我吧！以上。」\n###\n春日:「找出外星人、未来人以及超能力者，然后跟他们一起玩!」\n古泉:「啊，原来如此。」\n古泉:「真不愧是凉宫同学。」\n古泉:「没问题，我加入，今后还请大家多多指教。」\n古泉:「我是古泉。因为才刚转学过来，所以会有很多不懂的地方，还请您多多指教。」\n阿虚:「啊，我是……」\n春日:「他叫阿虚。」\n春日:「那个长得很可爱的是实玖瑠，另一个戴眼镜的女生是有希。」\n春日:「好了，这样我们就有五个人了。这么一来，学校就无话可说了!」\n###\n春日:「怎么都没发生学生一个接一个失踪，或者老师在形成密室的教室里遭遇到杀害这种刺激的事啊? 」\n阿虚:「你别说那种吓死人的话啦!」\n春日:「我参加过推理研究会喔。」\n阿虚:「咦!然后呢? 」\n春日:「真是笑死人了!直到目前为止，根本没半件像样的事件出现。而且，社员都只是些侦探推理小说迷而已，根本

让我们把这个放置到ChatHaruhi的默认配置中

In [ ]:
from ChatHaruhi import ChatHaruhi
from ChatHaruhi.response_openai import get_response

chatbot = ChatHaruhi( role_name = "于谦", llm = get_response )

response = chatbot.chat( user = "", text = "郭德纲: 听说现在人工智能发展得快啊" )
print( response )


Generating train split: 0 examples [00:00, ? examples/s]

query_rags: [{'n': 5, 'max_token': 1000, 'query': '郭德纲: 听说现在人工智能发展得快啊', 'lid': 11}] rest_limit = 1447
于谦: 是啊，现在的科技发展，真是越来越厉害了。人工智能也是个大热门啊。
